In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
import lxml
import time
import os
# Base URL pattern for dream moods dictionary


# List to store dream symbols and meanings
dreams = []
meanings = []

In [ ]:


def download_letter(letter, suffix="{}_all.htm"):    
    base_url = "http://www.dreammoods.com/dreamdictionary/"

    file = f"html/{letter}_all.html"
    if os.path.exists(file):
        print (f"File {file} already exists, skipping download.")
        return
    
    url = base_url + suffix.format(letter)
   
    response = requests.get(url)


    if response.status_code == 200:  # Check if the page was successfully fetched
        with open(file, "w") as f:
            f.write(response.text)
        print(f"Downloaded {letter} to {file}")

    else:
        print(f"Failed to retrieve {letter}, trying alternative")
        return download_letter(letter, suffix="{}.htm")


for letter in string.ascii_lowercase:
    download_letter(letter)
    time.sleep(.5)  # Sleep for 1 second to avoid overwhelming the server

File html/a_all.html already exists, skipping download.
File html/b_all.html already exists, skipping download.
File html/c_all.html already exists, skipping download.
File html/d_all.html already exists, skipping download.
File html/e_all.html already exists, skipping download.
File html/f_all.html already exists, skipping download.
File html/g_all.html already exists, skipping download.
File html/h_all.html already exists, skipping download.
File html/i_all.html already exists, skipping download.
File html/j_all.html already exists, skipping download.
File html/k_all.html already exists, skipping download.
File html/l_all.html already exists, skipping download.
File html/m_all.html already exists, skipping download.
File html/n_all.html already exists, skipping download.
File html/o_all.html already exists, skipping download.
File html/p_all.html already exists, skipping download.
File html/q_all.html already exists, skipping download.
File html/r_all.html already exists, skipping do

In [72]:

import lxml.html as lhtml

import pandas as pd
import re

from plotly import express as px

In [73]:
def break_attr(dic):
    drop = ['style','align','face']
    for dr in drop:
        if dr in dic:
            dic.pop(dr)
    return str(dic)

def classify(t):
    if t.tag == "b":
        return "sym" #"bold"
    
    if t.tag == "strong":
        return "sym" #"bold"    

    elif t.tag == "font":
        if "cheating on your spouse mate" in str(t.text):
            print("found cheating")
        if 'size' in t.attrib:
            sz = t.attrib['size']
            if sz == "4":
                return "sym" #"size4"
            elif sz == "3":
                return "mean" #"size3"
        else:
            return "mean" # no size defined
            
    elif t.tag == "a":
        if 'href' in t.attrib:
            return "link"
        elif 'name' in t.attrib:
            return "sym" #"header"
    return "mean" # "desc"

def collect(txt):
    if txt == None:
        return ""
    else:
        txt = txt.replace("\n"," ")
        txt = txt.replace("\xa0"," ")
        txt = re.sub(r'\s+', ' ', txt)
        return txt

syms_data = []

def breakdown(r, level=0):
    res =""
    clas = ""
    cont = ""
    for c in r.getchildren():

        res +=  "tag:" + str(c.tag) + "|attr:" + break_attr(c.attrib) + " |text:" + str(c.text).replace("\n","_") + "\n"
        mres, clas, cont = breakdown(c,level+1)
        res += "    "*level + mres + "\n"
        
        clas += classify(c) + ","
        cont = collect(c.text) + cont
        if level == 0:
            #res += "class:" + clas + "\n"
            clas = "sym" if "sym" in clas else "mean"
            
            res += "class:" + clas + "\n"
            res += "cont:" + cont + "\n"
            global syms_data
            syms_data.append({"clas": clas, "cont": cont})
            res+= "======================\n"

    return res, clas, cont 



In [80]:
with open("html/a_all.html", "r") as f:
    html = f.read()

In [81]:

        
xp = "/html/body/table[2]/tr/td/div/center/table/tr[4]/td[1]"

tree = lhtml.fromstring(html)

#tree.text_content()

tabl = tree.xpath(xp)[0]

In [82]:
r,c,t = breakdown(tabl)
with open("sandbox.txt", "w") as f:
    f.write(r)


In [83]:
syms_data = pd.DataFrame(syms_data)
syms_data 

,clas,cont
0,sym,
1,sym,A
2,mean,TOP
3,sym,
4,sym,Aardvark
...,...,...
891,sym,Axle
892,mean,To see an axle in your dream suggests that you...
893,mean,To dream that you have a broken axle indicates...
894,mean,


In [84]:
def squeeze_meaning_by_symbol(syms_data):
    data2 =[]
    sym = ""
    meanings = []

    for i, r in syms_data.iterrows():
        if r['clas'] == "sym":
            
            data2.append({"symbol": sym, "meaning": meanings})
            meanings = []
            sym = r['cont']
        else:
            meanings.append(r['cont'])
        

    data2 = pd.DataFrame(data2)
    data2['n_meanings'] = data2.meaning.apply(len)
    return data2

data2 = squeeze_meaning_by_symbol(syms_data)    

In [85]:
data2

,symbol,meaning,n_meanings
0,,[],0
1,,[],0
2,A,[TOP],1
3,,[],0
4,Aardvark,[TOP],1
...,...,...,...
352,Avoid,[To dream that you are avoiding a person in yo...,1
353,Awaken,[To awaken in your dream represents a spiritua...,2
354,Awakening,[To dream that you have an awakening signifies...,2
355,Award,[To dream that you receive an award indicates ...,1


In [86]:
data3 = data2.explode('meaning')
data3['mean_len'] = data3.meaning.str.len()
data3

,symbol,meaning,n_meanings,mean_len
0,,NaN,0,NaN
1,,NaN,0,NaN
2,A,TOP,1,3.0
3,,NaN,0,NaN
4,Aardvark,TOP,1,3.0
...,...,...,...,...
354,Awakening,To dream that you have an awakening signifies ...,2,146.0
354,Awakening,You may also be on the verge of lucid dreaming...,2,51.0
355,Award,To dream that you receive an award indicates t...,1,201.0
356,Ax,To see an ax in your dream indicates that you ...,2,291.0


In [87]:
def extract_letter(file):
    with open(file, "r") as f:
        html = f.read()
        
    xp = "/html/body/table[2]/tr/td/div/center/table/tr[4]/td[1]"

    tree = lhtml.fromstring(html)

    #tree.text_content()

    tabl = tree.xpath(xp)[0]

    global syms_data
    syms_data = []
    r,c,t = breakdown(tabl)
    syms_data = pd.DataFrame(syms_data)
    data2 = squeeze_meaning_by_symbol(syms_data)
    data3 = data2.explode('meaning')
    data3['mean_len'] = data3.meaning.str.len()
    return data3



In [88]:
dfs = []
save_dir = 'html'

for f in os.listdir(save_dir):
    if f.endswith(".html"):
        try:
            existing_df = extract_letter(os.path.join(save_dir, f))
            existing_df["filename"] = f
            dfs.append(existing_df)
        except Exception:
            continue

dataset = pd.concat(dfs)
dataset

found cheating


,symbol,meaning,n_meanings,mean_len,filename
0,,NaN,0,NaN,a_all.html
1,,NaN,0,NaN,a_all.html
2,A,TOP,1,3.0,a_all.html
3,,NaN,0,NaN,a_all.html
4,Aardvark,TOP,1,3.0,a_all.html
...,...,...,...,...,...
373,Mutilate,To dream that you are being mutilated indicate...,2,431.0,m_all.html
373,Mutilate,To dream that someone or something is mutilate...,2,99.0,m_all.html
374,Muzzle,To see or use a muzzle in your dream suggests ...,1,203.0,m_all.html
375,Myrrh,To see myrrh in your dream signifies punishmen...,1,113.0,m_all.html


In [90]:
from utils import save_df_as_pretty_html
save_df_as_pretty_html(dataset, "rescraped.html")
dataset.to_csv("rescraped.csv", index=False)

✅ HTML table saved to: rescraped.html


In [91]:
px.histogram(dataset.mean_len)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'bingroup': 'x',
              'hovertemplate': 'variable=mean_len<br>value=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': 'mean_len',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'mean_len',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': {'bdata': ('AAAAAAAA+H8AAAAAAAD4fwAAAAAAAA' ... 'AAAABgaUAAAAAAAEBcQAAAAAAAAFZA'),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'value'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}}}
})

In [220]:
def clean_meaning_text(meaning_tag):
    meaning = meaning_tag.get_text(separator=" ", strip=True)  # Remove excessive whitespace and handle multi-line texts
    # Remove "TOP" links or other unwanted parts
    meaning = meaning.replace(" TOP", "").strip()
    return meaning

In [221]:
def extract_symbol(symbol):
    dream = symbol.get_text().strip()  # Get the dream symbol text

    # Find the corresponding meaning (look for the next <p> tag that contains the style attribute for consistency)
    meaning_tag = symbol.find_next('p', style=True)
    
    if meaning_tag:
        meaning = clean_meaning_text(meaning_tag)
        
        # Check if meaning and dream are valid
        if dream and meaning:
            return dream, meaning

    else:
        print(f"Could not find meaning for {dream}")

# original code

In [ ]:
for letter in string.ascii_lowercase:
    url = base_url.format(letter)
    response = requests.get(url)

    if response.status_code == 200:  # Check if the page was successfully fetched
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all dream symbols, wrapped inside <a> tags with a name attribute
        dream_symbols = soup.find_all('a', attrs={'name': True})
        
        # Iterate over each symbol
        for symbol in dream_symbols:
            dream, meaning = extract_symbol(symbol)
            dreams.append(dream)
            meanings.append(meaning)

    else:
        print(f"Failed to retrieve {letter}")

In [4]:
df = pd.DataFrame({
    "Dream Symbol": dreams,
    "Interpretation": meanings
})

In [5]:
df

,Dream Symbol,Interpretation
0,Aardvark,To\nsee an aardvark in your dream indicates th...
1,Abandonment,To\ndream that you are abandoned suggests that...
2,Abduction,To\ndream of being abducted indicates that you...
3,Aborigine,To\nsee an Aborigine in your dream represents ...
4,Abortion,To\ndream that you have an abortion suggests t...
...,...,...
897,Yoyo,To see or play\nwith a yoyo in you dream signi...
898,Zebra,To see a zebra in\nyour dream represents perfe...
899,Zip\nLine,To dream that you\nare zip lining implies that...
900,Zombie,To see or dream\nthat you are a zombie suggest...


In [6]:
# Clean up the "Interpretation" column by removing newline characters and extra spaces
df['Interpretation'] = df['Interpretation'].replace(r'\n', ' ', regex=True).str.strip()

# Display the first few cleaned rows to verify the changes
df.head()

,Dream Symbol,Interpretation
0,Aardvark,To see an aardvark in your dream indicates tha...
1,Abandonment,To dream that you are abandoned suggests that ...
2,Abduction,To dream of being abducted indicates that you ...
3,Aborigine,To see an Aborigine in your dream represents b...
4,Abortion,To dream that you have an abortion suggests th...


In [7]:
df.to_csv("dreams_interpretations.csv", index=False)

In [8]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [9]:
download_file("/kaggle/working/dreams_interpretations.csv", "download")

/kaggle/working/download.zip